<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/streaming_tts_repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q git+https://github.com/coqui-ai/tts.git@9175a4d8d85815e24956e22aa76590f96e2815d9 deepspeed==0.8.3 gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
import torch
import torchaudio
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.generic_utils import get_user_data_dir

os.environ["COQUI_TOS_AGREED"] = "1"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1")

model_path = os.path.join(get_user_data_dir("tts"), "tts_models--multilingual--multi-dataset--xtts_v1")
config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))
model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=os.path.join(model_path, "model.pth"),
    vocab_path=os.path.join(model_path, "vocab.json"),
    eval=True,
    use_deepspeed=True
)
model.cuda()

gpt_cond_latent, _, speaker_embedding = model.get_conditioning_latents(audio_path="female.wav")
wav_chuncks = []

FileNotFoundError: ignored

In [ ]:
chunks = model.inference_stream(
    "It took me quite a long time to develop a voice and now that I have it I am not going to be silent.",
    "en",
    gpt_cond_latent,
    speaker_embedding
)

for i, chunk in enumerate(chunks):
    wav_chuncks.append(chunk)
    print(f"Received chunk {i} of audio length {chunk.shape[-1]}")
wav = torch.cat(wav_chuncks, dim=0)
torchaudio.save("xtts_streaming.wav", wav.squeeze().unsqueeze(0).cpu(), 24000)

# Gradio app test

In [ ]:
import os
import torch
import torchaudio
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.generic_utils import get_user_data_dir

import gradio as gr

os.environ["COQUI_TOS_AGREED"] = "1"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1")

model_path = os.path.join(get_user_data_dir("tts"), "tts_models--multilingual--multi-dataset--xtts_v1")
config = XttsConfig()
config.load_json(os.path.join(model_path, "config.json"))
model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=os.path.join(model_path, "model.pth"),
    vocab_path=os.path.join(model_path, "vocab.json"),
    eval=True,
    use_deepspeed=True
)
model.cuda()

def stream_audio(synthesis_text):
    gpt_cond_latent, _, speaker_embedding = model.get_conditioning_latents(audio_path="female.wav")
    wav_chunks = []

    chunks = model.inference_stream(
        synthesis_text,
        "en",
        gpt_cond_latent,
        speaker_embedding)

    for i, chunk in enumerate(chunks):
        wav_chunks.append(chunk)
        print(f"Received chunk {i} of audio length {chunk.shape[-1]}")
        yield chunk.detach().cpu().numpy().squeeze()

demo = gr.Interface(
    fn=stream_audio,
    inputs=gr.Textbox(),
    outputs=gr.Audio(autoplay=True, streaming=True),
)

if __name__ == "__main__":
    demo.queue().launch(debug=True)

 > tts_models/multilingual/multi-dataset/xtts_v1 is already downloaded.
 > Using model: xtts
